In [52]:
""" Simplify geometry
-------------------------------------------------------------------------------

Simplify hydrobasin level 6 and 7 geometries. 

This script is not used since geopandas creates gaps between polygons after 
simplification. MapShaper can use a more advanced simplification algorithm.

We use https://mapshaper.org/ with the following settings:

- input_files = See S3 input path
- detect line intersections = True
- snap vertices = True

Simplify
- prevent shape removal = False
- use planar geometry = False
- method: Visvalingam's Algorithm / weighted area

percentage: 50% (Selected based on visual inspection)
44 line errors (level7)
12 line errors (level6):
- repair = True


Author: Rutger Hofste
Date: 20190613
Kernel: python35
Docker: rutgerhofste/gisdocker:ubuntu16.04

"""

SCRIPT_NAME = "Y2019M06D13_RH_Simplify_Geometries_V01"
OUTPUT_VERSION  = 1
 
# Tolerance in degrees (appr. 1 km at equator)
TOLERANCE = 360/43200 

S3_INPUT_PATH= "s3://wri-projects/Aqueduct30/rawData/WWF/HydroSheds30sComplete/00_consolidated"

ec2_input_path = "/volumes/data/{}/input_V{:02.0f}".format(SCRIPT_NAME,OUTPUT_VERSION)
ec2_output_path = "/volumes/data/{}/output_V{:02.0f}".format(SCRIPT_NAME,OUTPUT_VERSION)

s3_output_path = "s3://wri-projects/Aqueduct30/processData/{}/output_V{:02.0f}/".format(SCRIPT_NAME,OUTPUT_VERSION)


In [32]:
import time, datetime, sys
dateString = time.strftime("Y%YM%mD%d")
timeString = time.strftime("UTC %H:%M")
start = datetime.datetime.now()
print(dateString,timeString)
sys.version
%matplotlib inline

Y2019M06D13 UTC 19:40


In [7]:
levels = [6]

In [17]:
level = levels[0]

In [9]:
!rm -r {ec2_input_path} 
!rm -r {ec2_output_path} 
!mkdir -p {ec2_input_path} 
!mkdir -p {ec2_output_path} 

In [18]:
s3_input_path = "{}/level{}".format(S3_INPUT_PATH,level)

In [19]:
s3_input_path

's3://wri-projects/Aqueduct30/rawData/WWF/HydroSheds30sComplete/00_consolidated/level6'

In [20]:
!aws s3 cp {s3_input_path} {ec2_input_path} --recursive

download: s3://wri-projects/Aqueduct30/rawData/WWF/HydroSheds30sComplete/00_consolidated/level6/hybas_merged_standard_level6_V01.cpg to ../../../data/Y2019M06D13_RH_Simplify_Geometries_V01/input_V01/hybas_merged_standard_level6_V01.cpg
download: s3://wri-projects/Aqueduct30/rawData/WWF/HydroSheds30sComplete/00_consolidated/level6/hybas_merged_standard_level6_V01.prj to ../../../data/Y2019M06D13_RH_Simplify_Geometries_V01/input_V01/hybas_merged_standard_level6_V01.prj
download: s3://wri-projects/Aqueduct30/rawData/WWF/HydroSheds30sComplete/00_consolidated/level6/hybas_merged_standard_level6_V01.sbx to ../../../data/Y2019M06D13_RH_Simplify_Geometries_V01/input_V01/hybas_merged_standard_level6_V01.sbx
download: s3://wri-projects/Aqueduct30/rawData/WWF/HydroSheds30sComplete/00_consolidated/level6/mergeShapefilesArc01.py to ../../../data/Y2019M06D13_RH_Simplify_Geometries_V01/input_V01/mergeShapefilesArc01.py
download: s3://wri-projects/Aqueduct30/rawData/WWF/HydroSheds30sComplete/00_consol

In [21]:
input_filename = "hybas_merged_standard_level{}_V01.shp".format(level)

In [23]:
input_path = "{}/{}".format(ec2_input_path,input_filename)

In [24]:
import geopandas as gpd

In [25]:
gdf = gpd.read_file(filename=input_path)

In [28]:
gdf =gdf[1:10]

In [40]:
gdf_simplified = gdf.copy()

In [41]:
gdf_simplified["geometry"] = gdf.geometry.simplify(tolerance=TOLERANCE,preserve_topology=True)

In [45]:
output_filename = "hydrobasin_level_{:02d}_simplified.shp".format(level)

In [47]:
output_path = "{}/{}".format(ec2_output_path,output_filename)

In [54]:
gdf.crs

{'init': 'epsg:4326'}

In [51]:
gdf_simplified.to_file(filename=output_path,
                       driver="ESRI Shapefile")

In [53]:
!aws s3 cp {ec2_output_path} {s3_output_path} --recursive 

upload: ../../../data/Y2019M06D13_RH_Simplify_Geometries_V01/output_V01/hydrobasin_level_06_simplified.prj to s3://wri-projects/Aqueduct30/processData/Y2019M06D13_RH_Simplify_Geometries_V01/output_V01/hydrobasin_level_06_simplified.prj
upload: ../../../data/Y2019M06D13_RH_Simplify_Geometries_V01/output_V01/hydrobasin_level_06_simplified.shx to s3://wri-projects/Aqueduct30/processData/Y2019M06D13_RH_Simplify_Geometries_V01/output_V01/hydrobasin_level_06_simplified.shx
upload: ../../../data/Y2019M06D13_RH_Simplify_Geometries_V01/output_V01/hydrobasin_level_06_simplified.dbf to s3://wri-projects/Aqueduct30/processData/Y2019M06D13_RH_Simplify_Geometries_V01/output_V01/hydrobasin_level_06_simplified.dbf
upload: ../../../data/Y2019M06D13_RH_Simplify_Geometries_V01/output_V01/hydrobasin_level_06_simplified.cpg to s3://wri-projects/Aqueduct30/processData/Y2019M06D13_RH_Simplify_Geometries_V01/output_V01/hydrobasin_level_06_simplified.cpg
upload: ../../../data/Y2019M06D13_RH_Simplify_Geometries